Fluid Dynamics and Godunov's Method
In this class, we will solve Euler's fluid equation using the Godunov Method

We will start with 1-D problem, trying different Riemann solver and reconstruction method.

1-D Godunov's method
We have five variables:

conservative variable (rho, mx1, mx2, mx3, E) or (U[igrid, 0], ..., U[igrid, 4])

primitive variable (rho, vx1, vx2, vx3, pressure) or (W[igrid, 0], ..., W[igrid, 4])

Step 1: initialize the problem

Step 2: compute dt based on CFL condition

Step 3: for each active zone, reconstruct its left and right states

Step 4: for each active zone, use Reimann solver to calculate the flux

Step 5: update quantities using the flux $$
U&lt;em&gt;{j}^{n+1}=U&lt;/em&gt;{j}^{n}+(F&lt;em&gt;{j-1/2}-F&lt;/em&gt;{j+1/2})&lt;em&gt;area/volume&lt;/em&gt;dt
$$ Note that U have 5 varaibles

Step 6: conversion from conservative to primitive variables. Primitive variables are useful for timestep and boundary condition calculations

In [1]:
# 1-D Godunov Scheme, simplest possible

import numpy as np
import matplotlib.pylab as p
from mpl_toolkits.mplot3d import Axes3D 

L=100.; Nx=100; NGHOST=2 
tend=21.; vel=1.; Dtout=10.

Nxg=Nx+2*NGHOST  # Total grid number including ghost zones
ist=2           # active zone starting grid number
ien=Nx+2        # active zone ending grid number

NHYDRO=5 # 5 varialbes, rho, mx1, mx2, mx3, E, or rho, vx1, vx2, vx3, pressure
gamma=1.4

dx=L/Nx
darea=1.
dvol=darea*dx

Wsave=np.zeros((Nxg,NHYDRO), float) # save the data for later comparison

def main():

    U=np.zeros((Nxg,NHYDRO), float)  # conserved variable
    W=np.zeros((Nxg,NHYDRO), float)  # primative variable
    Flux=np.zeros((Nxg,NHYDRO), float) 
    fluxa=np.zeros(NHYDRO,float)
    wl=np.zeros(NHYDRO, float)
    wr=np.zeros(NHYDRO, float)

    #Step 1: initialize the problem
    
    for i in range(int((Nxg-1)/2)):
        U[i,0]=8.
        W[i,0]=8.
        W[i,4]=10./gamma
        U[i,4]=W[i,4]/(gamma-1) 
    
    for i in range(int((Nxg-1)/2), Nxg):
        U[i,0]=1.
        W[i,0]=1.
        W[i,4]=1.0/gamma
        U[i,4]=W[i,4]/(gamma-1) 

    toutn=0
    t=0.
    while t < tend:           
        
        #Step 2: calculate dt
        
        dt=0.5*np.min(dx/(np.sqrt(W[ist:ien,1]*W[ist:ien,1]+W[ist:ien,2]*W[ist:ien,2]+W[ist:ien,3]*W[ist:ien,3])+np.sqrt(gamma*W[ist:ien,4]/W[ist:ien,0])))

        for ix in np.arange(ist, ien+1):                       
            # Step 3: reconstruction for left and right state
            reconst_donor(W, wl, wr, ix, dx)       # reconstruction
            # Step 4: Reimann solver to calculate the flux
            rsolver(wl,wr,fluxa, ix)               # Reimann solver
            Flux[ix,:]=fluxa[:]                    # Copy flux
        for ix in np.arange(ist, ien):
            # Step 5: use a conservative form to update the quantities
            for idex in range(NHYDRO):
                # Please finish the next line
                U[ix,idex]=   #update conservative variables
        for ix in np.arange(ist, ien): 
            # Step 6: conversion from conservative to primitive variables
            contoprim(U, W, ix)                    # conservative to primitive variables
        if t >= toutn*Dtout: 
            p.plot(W[:,0])
            print(toutn)   
            toutn += 1                        
        t+=dt
    Wsave[:,:]=W[:,:]
    
    def reconst_donor(W, wl, wr, ix, dl): # donor cell reconstruction
    for idex in range(NHYDRO):
        wl[idex]=W[ix-1,idex]
        wr[idex]=W[ix,idex]

def rsolver(wl,wr,fluxa, ix):
    # Compute Roe-averaged state
    sqrtdl = np.sqrt(wl[0])   
    sqrtdr = np.sqrt(wr[0])
    isdlpdr = 1./(sqrtdl+sqrtdr)
    
    roed=sqrtdl*sqrtdr
    roevx=(sqrtdl*wl[1]+sqrtdr*wr[1])*isdlpdr
    roevy=(sqrtdl*wl[2]+sqrtdr*wr[2])*isdlpdr
    roevz=(sqrtdl*wl[3]+sqrtdr*wr[3])*isdlpdr
        #The enthalpy H=(E+P)/d is averaged sqrtdl*hl = sqrtdl*(el+pl)/dl = (el+pl)/sqrtdl
    el = wl[4]/(gamma-1.)+0.5*wl[0]*(wl[1]*wl[1]+wl[2]*wl[2]+wl[3]*wl[3])
    er = wr[4]/(gamma-1.)+0.5*wr[0]*(wr[1]*wr[1]+wr[2]*wr[2]+wr[3]*wr[3])
    roeh = ((el+wl[4])/sqrtdl+(er+wr[4])/sqrtdr)*isdlpdr
    
    # Compute sound speed in L,R, and Roe-averaged state
    cl=np.sqrt(gamma*wl[4]/wl[0])
    cr=np.sqrt(gamma*wr[4]/wr[0])
    q=roeh-0.5*(roevx*roevx+roevy*roevy+roevz*roevz)
    if(q<0.0):
        q=0.0
    a=np.sqrt((gamma-1.)*q)

    # Compute the max/min wave speeds based on L/R and Roe-averaged values
    al=min([roevx-a,wl[1]-cl])
    ar=max([roevx+a,wr[1]+cr])
    if ar>0.0:
        bp=ar
    else:
        bp=0.0
    if al<0.0:
        bm=al
    else:
        bm=0.0
        
    #Compute L/R fluxes along the lines bm/bp: F_L - (S_L)U_L; F_R - (S_R)U_R
    fl=np.zeros(NHYDRO, float)
    fr=np.zeros(NHYDRO, float)
    
        fl[0]=wl[0]*wl[1]-bm*wl[0]
    fr[0]=wr[0]*wr[1]-bp*wr[0]
    fl[1]=wl[0]*wl[1]*(wl[1]-bm)
    fr[1]=wr[0]*wr[1]*(wr[1]-bp)
    fl[2]=wl[0]*wl[2]*(wl[1]-bm)
    fr[2]=wr[0]*wr[2]*(wr[1]-bp)
    fl[3]=wl[0]*wl[3]*(wl[1]-bm)
    fr[3]=wr[0]*wr[3]*(wr[1]-bp)
    fl[1] += wl[4]
    fr[1] += wr[4]
    fl[4] = el*(wl[1]-bm)+wl[4]*wl[1]
    fr[4] = er*(wr[1]-bp)+wr[4]*wr[1]
    
    if (bp==bm):
        tmp = 0.0
    else:
        tmp = 0.5*(bp+bm)/(bp-bm)
        
    fluxa[0]=0.5*(fl[0]+fr[0])+(fl[0]-fr[0])*tmp
    fluxa[1]=0.5*(fl[1]+fr[1])+(fl[1]-fr[1])*tmp
    fluxa[2]=0.5*(fl[2]+fr[2])+(fl[2]-fr[2])*tmp
    fluxa[3]=0.5*(fl[3]+fr[3])+(fl[3]-fr[3])*tmp
    fluxa[4]=0.5*(fl[4]+fr[4])+(fl[4]-fr[4])*tmp
    
def contoprim(U, W, ix):    # conservative variables (mx, my, mz, E) to primative variables (vx, vy, vz, pressure)
    W[ix,0]=U[ix,0]
    W[ix,1]=U[ix,1]/W[ix,0]
    W[ix,2]=U[ix,2]/W[ix,0]
    W[ix,3]=U[ix,3]/W[ix,0]
    W[ix,4]=(U[ix,4]-0.5*W[ix,0]*(W[ix,1]**2+W[ix,2]**2+W[ix,3]**2))*(gamma-1)

SyntaxError: invalid syntax (<ipython-input-1-9e8cd53f0254>, line 64)